In [6]:
import torch
import torchvision
import torchvision.transforms as transforms
import sys

import d2lzh_pytorch as d2l

读取数据

这个jupyterlab太多包没有了（自己的电脑上并没有出现这个问题）...为了方便重写函数

根据官方教程，windows系统上num_workers设置为大于1会出现问题：  
If running on Windows and you get a BrokenPipeError, try setting
the num_worker of torch.utils.data.DataLoader() to 0.

In [7]:
def load_fmnist_dataset(batch_size):
    mnist_train = torchvision.datasets.FashionMNIST(root='~/Datasets/FashionMNIST', train=True, download=True, transform=transforms.ToTensor())
    mnist_test = torchvision.datasets.FashionMNIST(root='~/Datasets/FashionMNIST', train=False, download=True, transform=transforms.ToTensor())
    
    if sys.platform.startswith('win'):  
        num_workers = 0
    else:
        num_workers = 4
    
    train_iter = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    test_iter = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    
    return train_iter, test_iter

In [8]:
batch_size = 256

train_iter, test_iter = load_fmnist_dataset(batch_size)

初始化模型参数

In [9]:
num_input = 28 * 28
num_output = 10

W = torch.normal(0, 0.01, size=(num_input, num_output), dtype=torch.float)
b = torch.zeros(num_output, dtype=torch.float)

设置模型参数梯度

In [10]:
W.requires_grad_(requires_grad=True)
b.requires_grad_(requires_grad=True)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)

实现softmax运算

首先了解torch.sum()的具体使用方法

In [11]:
X = torch.tensor([[1, 2, 3], [4, 5, 6]])

# dim=0，按行求和的意思是：结果的行数->1，不同行同一列元素之间求和
print(X.sum(dim=0, keepdim=True))
# dim=1，按列求和，则是同一行不同列元素求和
print(X.sum(dim=1, keepdim=True))

# keepdim=False，二维张量求和后自动转化为一维张量，其他类似
print(X.sum(dim=0, keepdim=False))
print(X.sum(dim=1, keepdim=False))

tensor([[5, 7, 9]])
tensor([[ 6],
        [15]])
tensor([5, 7, 9])
tensor([ 6, 15])


使用函数定义softmax运算

我们的W定义的是784X10，因此样本是28X28也就应该转化为1X784，最后运算得到的结果为1X10，1行10列，因此应该令dim=1求和

In [12]:
def softmax(X):
    X_exp = X.exp()
    partition = X_exp.sum(dim=1, keepdim=True)
    return X_exp / partition  # 此处用到广播机制

用一个小例子看看我们定义的函数是否正确

In [13]:
X = torch.rand((2, 5))

X_prob = softmax(X)

print(X)
print(X_prob, X_prob.sum(dim=1))

tensor([[0.8467, 0.3811, 0.4029, 0.5119, 0.8394],
        [0.8183, 0.4189, 0.2610, 0.6749, 0.2511]])
tensor([[0.2514, 0.1578, 0.1613, 0.1799, 0.2496],
        [0.2720, 0.1824, 0.1558, 0.2356, 0.1542]]) tensor([1., 1.])


定义softmax回归的模型

In [14]:
def net(X):
    return softmax(torch.mm(X.view(-1, num_input), W) + b)

定义损失函数

这里使用gather()来实现交叉熵。这里，如何理解gather()这个函数是非常重要的问题。

torch.gather(input, dim, index, out=None, sparse_grad=False) → Tensor

官方解释是：Gathers values along an axis specified by dim.

dim代表指定的维度。最终输出结果的size应该与index完全相同。

简单理解为：最终返回一个与index的size相同的tensor，其中dim指定的维度的index被gather中的index替代，其他维不变

In [15]:
y_hat = torch.tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
y = torch.LongTensor([0, 2])

y_hat.gather(1, y.view(-1, 1))

tensor([[0.1000],
        [0.5000]])

以下定义交叉熵损失函数

In [16]:
def cross_entropy(y_hat, y):
    return - torch.log(y_hat.gather(1, y.view(-1, 1)))

引入评价指标：准确率

y_hat.argmax(dim=1)返回每行中最大的值，做判断后得到一个ByteTensor，因此需要求float()转化为浮点数

In [17]:
def accuracy(y_hat, y):
    return (y_hat.argmax(dim=1) == y).float().mean().item()

In [18]:
print(accuracy(y_hat, y))

0.5


评价模型net在data_iter上的表现

In [19]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0
    for X, y in data_iter:
        acc_sum += (net(X).argmax(dim=1) == y).float().sum().item()
        n += y.shape[0]
    return acc_sum / n

net是随机初始化的，因此预测概率相当于随机猜猜对的概率（0.1）

In [20]:
print(evaluate_accuracy(test_iter, net))

0.0657
